# クラスタリング：関連のある文書を見つける

訓練データに各データに属するクラスがラベル付されており、その訓練データからモデルを学習することができる。これは「教師あり学習」と呼ばれている。今回はそういったラベルが付いていないデータに対して分類モデルを構築する方法を考えていくことになる。

ここではQ&Aサイトを運営する場面を想定して、現在見ているページの内容と関連する情報を提示することを考える。この問題に対するアプローチとして、「ある投稿された文書に対して、それ以外の文書との類似度をすべて算出する」事になるでしょう。その後「上位N個の文章に関するリンクをページ下部に表示する」といったことを行えばいいのではないだろうか。しかしながらこのアプローチでは、ページ数が増えてしまうと計算が追いつかなくなってしまう。そういったことを踏まえて、関連している文書を**素早く**見つける方法を見つけていく。

これは**クラスタリング（clustering）**を用いることで解決できる。クラスタとはデータ集合の部分集合であり、クラスタリングとは似ているもの同士を同じクラスタに分類する手法を指している。

## 文書の関連性を計測する

機械学習においては、テキストデータだけでは分析が出来ないため、データを数字に変換することが必要になってくる。

### やってはいけないこと

テキストデータの類似度を求めるために、レーベンシュタイン距離（Levenshtein distance）を利用することができる。レーベンシュタイン距離は**編集距離**と呼ばれることもある。２つの単語があった場合、一方の単語をもう一方の単語と同じアルファベットの並びにするために、編集を行う最小回数をいう。

たとえば「machine」と「mchiene」という単語があったとすると、この場合編集距離は*2*になる。すなわち、

1. mの後にaを挿入する
2. 最初のeを削除する

こういったアルゴリズムは計算コストがとても高くなる。

文章内の単語を最小単位として扱うことにして、文章全体の編集距離を考えてみる。ここでは文書1*「How to format my hard disk」*と文書2*「Hard disk format problems」*という文章の編集距離を考えてみる。

1. howを削除する
2. toを削除する
3. formatを削除する
4. myを削除する
5. formatを追加する
6. problemsを追加する

この場合変数距離は*6*とすることができる。これらは単語の編集距離よりかはコストを削減できていそうだが、時間のオーダは同じであるため、問題が残ってしまう。

### どうやるべきか

編集距離よりもロバストな手法として、**bag-of-words**と呼ばれるアプローチがある。この手法は単語の出現回数を特徴量として利用する。

文書1と文書2について、単語の出現回数は下のようになる。

|単語      |文書1での出現回数|文書2での出現回数|
|:---------|:---------------:|:---------------:|
| disk     | 1               | 1               |
| format   | 1               | 1               |
| now      | 1               | 0               |
| hard     | 1               | 1               |
| my       | 1               | 0               |
| problems | 0               | 1               |
| to       | 1               | 0               |

文書1と文書2の列をベクトルとして扱うことができる。このようにベクトル化することによってデータ・セット中のすべての文書館でユークリッド距離を計算することができ、最近傍点を見つけることができる。

クラスタリングの流れは次のようになる。

1. 各文書から特徴量を抽出し、特徴ベクトルの形で保存する。
2. 特徴ベクトルに対して、クラスタリングを行う。
3. 投稿された質問文書に対して、クラス決定する。
4. このクラスタに属する文書を他にいくつか集める。これで多様性を増やせる。

## 前処理：共通する単語の出現回数を類似度として計測する

bag-of-wordは高速に処理することができるが、いくつか問題点も存在する。

### テキストデータをbag-of-wordに変換する

ここではScikitのCountVectorizerを利用して単語の出現回数を数え、それをベクトルで表記できるようにする。

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=1)

*min_df*というパラメータは頻繁に使われ得ていない単語をCountVectorizerが無視するときに利用する。整数を渡せばその数より出現頻度の少ない単語は無視される。

生成したインスタンスをprintしてみると、Scikitがデフォルトのパラメータとしてどういった値を設定しているのかを確認することができる。

In [3]:
print(vectorizer)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)


*analyzer=word*と指定されている通り、これは単語レベルで出現回数が数えられていることを示している。

In [4]:
contents = ['How to format my hard disk', 'Hard disk format ploblem']
X = vectorizer.fit_transform(contents)
vectorizer.get_feature_names()

['disk', 'format', 'hard', 'how', 'my', 'ploblem', 'to']

*vectorizer*は7つの単語を、出現回数を数えるべき単語としてトークンとしている。

In [5]:
print(X.toarray().transpose())

[[1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]]


これは単語の出現回数の表と同じ結果になっていることが分かる。

### 単語を数える

ここでは*sample_documents*ディレクトリ下にある文書をデータセットとして分析を行ってみる。 [03.txtについて補足](supplemental.ipynb)

In [6]:
import os

SAMPLE_DOC_DIR = "sample_documents"
posts = [open(os.path.join(SAMPLE_DOC_DIR, f)).read() for f in sorted(os.listdir(SAMPLE_DOC_DIR))]

for num, post in enumerate(posts):
    print("0%d.txt : %s" % (num + 1, post))

vectorizer = CountVectorizer(min_df=1)

01.txt : This is a toy post about machine learning. Actually, it contains not much interesting stuff.
02.txt : Imaging databases provide storage capabilities.
03.txt : Most imaging databases save images permanently.
04.txt : Imaging databases store data.
05.txt : Imaging databases store data. Imaging databases store data. Imaging databases store data.


*vectorizer*には対象となる文書データをすべて知らせる必要がある。

In [7]:
x_train = vectorizer.fit_transform(posts)
num_samples, num_features = x_train.shape
print("#samples: %d, #features: %d" % (num_samples, num_features))

#samples: 5, #features: 25


5つの文章から25個の単語が存在することを示している。

In [8]:
print(vectorizer.get_feature_names())

['about', 'actually', 'capabilities', 'contains', 'data', 'databases', 'images', 'imaging', 'interesting', 'is', 'it', 'learning', 'machine', 'most', 'much', 'not', 'permanently', 'post', 'provide', 'save', 'storage', 'store', 'stuff', 'this', 'toy']


新しい文書について、次のようにベクトル化することができる。

In [9]:
new_post = "imaging databases"
new_post_vec = vectorizer.transform([new_post])

**transform()**によって返されるベクトルは疎なベクトルである。そのため、疎ベクトルに適したデータ構造が結果として返される。ほとんどの要素が0であるため、すべての単語の出現回数をまとめて表記することは行えない。代わりに出現した単語のみの情報だけをデータに格納する。

In [10]:
print(new_post_vec)

  (0, 5)	1
  (0, 7)	1


**toarray()**を使うことで、特徴ベクトルのすべての要素を表示することができる。

In [11]:
print("%s , type=%s" % (new_post_vec.toarray(), type(new_post_vec.toarray())))

[[0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]] , type=<class 'numpy.ndarray'>


類似度を算出するために、新しい文書と他の既存の文書の間でユークリッド距離を計算することにする。

In [12]:
import scipy as sp

def dist_raw(v1, v2):
    delta = v1 - v2
    return sp.linalg.norm(delta.toarray())

ここで**norm()**はユークリッドノルム（ユークリッド距離）を計算している。定義したdist_raw()を用いることで、他のすべての文書に対して類似度を計算することができる。

In [13]:
import sys

# 引数にベクトルの距離を計算する関数を渡す
def print_doc_dists(dist_func):
    best_doc = None
    best_dist = sys.maxsize
    best_i = None

    for i in range(0, num_samples):
        post = posts[i]
    
        if post == new_post:
            continue

        post_vec = x_train.getrow(i)

        d = dist_func(post_vec, new_post_vec)
        print("=== Post %i with dist=%.2f: %s" % (i, d, post))

        if d < best_dist:
            best_dist = d
            best_i = i

    print("\n[Result]\nBest post is %i with dist=%.2f" % (best_i, best_dist))

# dist_raw()を使って距離を計算するように
print_doc_dists(dist_raw)

=== Post 0 with dist=4.00: This is a toy post about machine learning. Actually, it contains not much interesting stuff.
=== Post 1 with dist=1.73: Imaging databases provide storage capabilities.
=== Post 2 with dist=2.00: Most imaging databases save images permanently.
=== Post 3 with dist=1.41: Imaging databases store data.
=== Post 4 with dist=5.10: Imaging databases store data. Imaging databases store data. Imaging databases store data.

[Result]
Best post is 3 with dist=1.41


新しい文書*"imaging databases"*と最も似ていない文書は*文書0*である結果が出た。たしかに、これら2つの文書間には共通する単語は存在しない。

また、新しい文書は文書1とよく似ていることが分かる。しかしながら最も類似しているものは文書3である。文書1と文書3はともに新しい文書の単語を含むが、文書1のほうが1単語多い文章から構成されている。よって文書3のほうが類似度が高くなる。

新しい文書に対しての類似度は、その2つの文書で同じであるべきだ。しかしながら結果は異なっている。ここで文書3と文書4の特徴ベクトルを出力してみる。

In [14]:
print(x_train.getrow(3).toarray())
print(x_train.getrow(4).toarray())

[[0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]]
[[0 0 0 0 3 3 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0]]


単語の出現回数だけを特徴量として用いているようである。これは正規化する必要がある。

### 単語の出現回数ベクトルを正規化する

単語の出現回数からなるベクトルではなく、正規化したベクトルを返すようにdist_raw()を拡張してみる。

In [15]:
def dist_norm(v1, v2):
    v1_normalized = v1 / sp.linalg.norm(v1.toarray())
    v2_normalized = v2 / sp.linalg.norm(v2.toarray())

    delta = v1_normalized - v2_normalized
    
    return sp.linalg.norm(delta.toarray())

このdist_norm()を使って、類似度を計算してみる。

In [16]:
print_doc_dists(dist_norm)

=== Post 0 with dist=1.41: This is a toy post about machine learning. Actually, it contains not much interesting stuff.
=== Post 1 with dist=0.86: Imaging databases provide storage capabilities.
=== Post 2 with dist=0.92: Most imaging databases save images permanently.
=== Post 3 with dist=0.77: Imaging databases store data.
=== Post 4 with dist=0.77: Imaging databases store data. Imaging databases store data. Imaging databases store data.

[Result]
Best post is 3 with dist=0.77


今回の結果は文書3と文書4で同じ類似度が出ていることが分かる。

### 重要度の低い単語を取り除く

今度は新しい文書*「imaging databases」*と文書2を比較してみることにする。新しい文書には、*most*、*safe*、*images*、*permanently*が含まれていない。こういった単語は文書内では重要度がそれぞれ異なっている。*most*は分野に関係なく様々な文章に登場してくる。こういった単語を**ストップワード（stop word）**として、処理の対象外とすべきである。ストップワードとは頻繁に使われる単語、文書の分類に貢献しない単語のことである。

CountVectorizerでは、簡単にストップワードを登録することができる。

In [17]:
vectorizer = CountVectorizer(min_df=1, stop_words='english')

ここではenglishと指定することで、318個の単語をストップワードとして登録できる。

In [18]:
print(sorted(vectorizer.get_stop_words())[0:20])

['a', 'about', 'above', 'across', 'after', 'afterwards', 'again', 'against', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'amoungst']


ストップワードを利用すると、単語リストは全部で7つ減る。

In [19]:
x_train = vectorizer.fit_transform(posts)
num_samples, num_features = x_train.shape
print(vectorizer.get_feature_names())
new_post_vec = vectorizer.transform([new_post])

['actually', 'capabilities', 'contains', 'data', 'databases', 'images', 'imaging', 'interesting', 'learning', 'machine', 'permanently', 'post', 'provide', 'save', 'storage', 'store', 'stuff', 'toy']


またストップワードを用いることで、類似度は次のようになる。

In [20]:
print_doc_dists(dist_norm)

=== Post 0 with dist=1.41: This is a toy post about machine learning. Actually, it contains not much interesting stuff.
=== Post 1 with dist=0.86: Imaging databases provide storage capabilities.
=== Post 2 with dist=0.86: Most imaging databases save images permanently.
=== Post 3 with dist=0.77: Imaging databases store data.
=== Post 4 with dist=0.77: Imaging databases store data. Imaging databases store data. Imaging databases store data.

[Result]
Best post is 3 with dist=0.77


文章1と文書2は同じ類似度になっていることが分かる。

### ステミング（stemming）

意味的には同じ単語が、語形変化により異なる単語としてカウントされることである。例えば文書2では*imaging*と*images*が含まれる。これらは1つの単語としてカウントすべきである。

単語を特定の語幹（後の語形変化に対する基本形）へ変換するためには**Natural Language Toolkit（NLTK）**を利用する。

NLTKで*graphisc*、*imaging*、*imagination*といったものをステミングしてみる。

In [21]:
import nltk.stem

s = nltk.stem.SnowballStemmer('english')

print(s.stem("graphics"))
print(s.stem("imaging"))
print(s.stem("imagination"))
print(s.stem("imagine"))

graphic
imag
imagin
imagin


動詞については、次のような結果になる。

In [22]:
print(s.stem("buys"))
print(s.stem("buying"))
print(s.stem("bought"))

buy
buy
bought


ステミングの結果は、必ずしも正しい英単語になるとは限らない。

#### NLTKのステマーを用いて、ベクトル化を拡張する

文書をCountVectorizerに入れる前に、ステミングを行う必要がある。ここでは*build_analyzer()*をオーバーライドして処理を追加してみることにする。

In [23]:
english_stemmer = nltk.stem.SnowballStemmer('english')

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: (english_stemmer.stem(w) for w in analyzer(doc))

vectorizer = StemmedCountVectorizer(min_df=1, stop_words='english')

ここでは、次の3つのことを行っている

* 前処理の段階で、文書を小文字に変換する
* トークン化の段階で、すべての単語を抜き出す
* それぞれの単語をステム化された単語に変換する

In [24]:
x_train = vectorizer.fit_transform(posts)
num_samples, num_features = x_train.shape
print(vectorizer.get_feature_names())
new_post_vec = vectorizer.transform([new_post])

['actual', 'capabl', 'contain', 'data', 'databas', 'imag', 'interest', 'learn', 'machin', 'perman', 'post', 'provid', 'save', 'storag', 'store', 'stuff', 'toy']


ステミングを行った後で、文書をベクトル化すると、imagingとimagesが同じ単語としてカウントされていることが分かる。

print_doc_dists(dist_norm)

### TF-IDFを用いる

これまで考えてきた特徴量は、文書に特定の単語が何回表れるか、その回数を数えただけの単純なものであった。これは、文章中に特定の単語が多く存在すればするほど、その単語に対する重要度が高くなることを意味している。

これは**「ある単語に対して対象の文書中で出現した回数をカウントするのに加えて、その単語が*他の文書*でどれだけ出現するのかをカウントし、その回数で割る（除算する）」**という方法で対応することができる。この方法を**TF-IDF（term frequency - inverse document frequency）**で実現することができる。

TF-IDFは、単語の出現頻度$TF$と逆文書頻度$IDF$の2つの指標に基づいて計算することができる。単語$t_i$の文書$d_j$における出現回数$n_{ij}$、文書$d_j$におけるすねての単語の出現回数の和$\sum_{k}{} n_{kj}$、総文書数$|D|$、単語$t_i$を含む文書数$|{d:d \ni t_i}|$とすると、

$$
    TF_{ij} = \frac{n_{ij}}{\sum_{k}{}n_{kj}}
$$

$$
    IDF_i = \log \frac{|D|}{|{d:d \ni t_i}|}
$$

$$
    TF-IDF = TF \cdot IDF
$$

のように計算することができる。

In [25]:
import scipy as sp
import math

def tf_idf(term, doc, docset):
    tf = float(doc.count(term)) / sum(doc.count(w) for w in set(doc))
    idf = math.log(float(len(docset)) / (len([doc for doc in docset if term in doc])))

    return tf * idf

3つの文書からなるデータセットを対象として、TF-IDFがどのように計算されるかを確認してみる。

In [26]:
doc_a, doc_abb, doc_abc = ["a"], ["a", "b", "b"], ["a", "b", "c"]
D = [doc_a, doc_abb, doc_abc]
print(tf_idf("a", doc_a, D))
print(tf_idf("b", doc_abb, D))
print(tf_idf("a", doc_abc, D))
print(tf_idf("b", doc_abc, D))
print(tf_idf("c", doc_abc, D))

0.0
0.27031007207210955
0.0
0.13515503603605478
0.3662040962227032


結果から"a"という単語は全ての文書で用いられているため、なんの意味も持たないことが分かる。また"b"という単語はdoc_abbという文書にとって、doc_abc文書の2倍重要であることが分かる。

TF-IDFを行う機能を持ったステマ―を以下のように実装することができる。

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

class StemmedTfidfVectorizer(TfidfVectorizer):
    def build_analyzer(self):
        analyzer = super(TfidfVectorizer, self).build_analyzer()
        return lambda doc: (english_stemmer.stem(w) for w in analyzer(doc))

vectorizer = StemmedTfidfVectorizer(min_df=1, stop_words='english')

特徴ベクトルの値は、単語の出現回数ではなく、TF-IDFの値がそれぞれの単語ごとに格納されるようになった。

### ここまでやってきたこと

テキストデータに対して以下のような前処理を行った。

1. テキストデータをトークン化する。
2. 頻出しすぎる単語は、関連する文書を見つけるために役立たないため、取り除く。
3. めったに使われない単語は、新しい文書でも使われる可能性が低いため、取り除いた。
4. 残った単語についてその出現回数をカウントする。
5. 文書全体の状況を考慮するため、単語の出現回数からTF-IDFを計算する。

bag-of-wordsによるアプローチは、優れた性能を発揮するが、次のような欠点も存在している。

* 単語の関連性について考慮されていない
* 否定的な意味を正しく捉えることができていない
* タイプミスに対応できていない